In [12]:
from math import *
import math
import serial
from fer import FER
import time
s = serial.Serial('COM3', 115200, timeout=1)
detector = FER()

In [8]:
s.close()

In [13]:
import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)
present = time.time()
while cap.isOpened():
    success, image = cap.read()
    
    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # To improve performance
    image.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(image)
    
    # To improve performance
    image.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image2 = image
    emotion = detector.detect_emotions(image)
    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 8000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
            
            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)

            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)

            # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]])

            # The Distance Matrix
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)
            
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
            
            x = round(angles[0] * 720, 2)
            y = round(angles[1] * 720, 2)
            z = round(angles[2] * 720, 2)
            # print(y)

            # See where the user's head tilting
            textx = f"pitch: {x}"
            texty = f"yaw: {y}"
            textz = f"roll: {z}"
            # Display the nose direction
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
            aSerial = f'{int(-y)} {int(x)} {int(z)}'
            #print(aSerial.encode(), end='\r')
            
            if(time.time()-present >= 1.2):
                s.write(aSerial.encode())
                s.readline()
                present = time.time()
            #time.sleep(1)
            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_3d_projection[0][0][0]), int(nose_3d_projection[0][0][1]))
            
            cv2.line(image, p1, p2, (255, 0, 0), 2)

            # Add the text on the image
            cv2.putText(image, textx, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, texty, (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, textz, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            for i in emotion:
                box = i["box"]
                cv2.rectangle(image2, (box[0],box[1]), (box[0]+box[2],box[1]+box[3]), (255, 0, 0), thickness=7)
                emotion_name, score = detector.top_emotion(image )
                bounding_box = i["box"]
                for index, (emotion_name, score) in enumerate(i['emotions'].items()):
                    color = (211, 211,211) if score < 0.01 else (255, 255, 0)
                    emotion_score = "{}: {}".format(emotion_name, "{:.2f}".format(score))

                    cv2.putText(image2,emotion_score,
                               (bounding_box[0], bounding_box[1] + bounding_box[3] + 30 + index * 15),
                               cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1,cv2.LINE_AA,)
            
                
    cv2.imshow('Head Pose Estimation', image)
    
    if cv2.waitKey(1) == 27: 
        break

cap.release()

In [31]:
import json

In [50]:
emotion[0]['emotions'].items()

dict_items([('angry', 0.05), ('disgust', 0.0), ('fear', 0.01), ('happy', 0.0), ('sad', 0.16), ('surprise', 0.0), ('neutral', 0.78)])

In [12]:
cv2.imshow('Emotion', image)

In [4]:
pip install fer --user

Note: you may need to restart the kernel to use updated packages.


ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


In [10]:
results.multi_face_landmarks

[landmark {
   x: 0.3912947177886963
   y: 0.7807265520095825
   z: -0.02319261245429516
 }
 landmark {
   x: 0.3904902935028076
   y: 0.7402278184890747
   z: -0.03735733404755592
 }
 landmark {
   x: 0.39162710309028625
   y: 0.7544248104095459
   z: -0.02110632136464119
 }
 landmark {
   x: 0.3864462077617645
   y: 0.7123220562934875
   z: -0.025305844843387604
 }
 landmark {
   x: 0.39061129093170166
   y: 0.7302315831184387
   z: -0.03904883563518524
 }
 landmark {
   x: 0.3913607895374298
   y: 0.7189196944236755
   z: -0.035542916506528854
 }
 landmark {
   x: 0.39388957619667053
   y: 0.6941658854484558
   z: -0.014808212406933308
 }
 landmark {
   x: 0.349911630153656
   y: 0.7003149390220642
   z: 0.01763204112648964
 }
 landmark {
   x: 0.395070344209671
   y: 0.6785932779312134
   z: -0.009013851173222065
 }
 landmark {
   x: 0.3954775929450989
   y: 0.6670176982879639
   z: -0.009208365343511105
 }
 landmark {
   x: 0.3976130187511444
   y: 0.6205021739006042
   z: 0.00330